In [ ]:
import requests
import pandas as pd
import json

# Use API

## Elastic Search (adddata)

In [ ]:
with open('../sa2.json', 'r') as file:
    age = json.load(file)

sa2_data = []
for feature in age['features']:
    sa2_code = feature['properties']['sa2_code_2021']
    median_age = feature['properties']['median_age_persons']
    sa2_data.append({'SA2_CODE21': sa2_code, 'median_age': median_age})

sa2_df = pd.DataFrame(sa2_data)
sa2_df

In [ ]:
sa2_list = sa2_df.to_dict('records')
for i, row in enumerate(sa2_list):
    row['id'] = i

print(len(sa2_list))
sa2_list

In [ ]:
res = requests.post(url='http://127.0.0.1:9090/adddata/sa2',
                    headers={'Content-Type': 'application/json'},
                    data=json.dumps(sa2_list))

print(res.status_code, res.text)

## Data Processing APIs
### Text Processing API

In [ ]:
text1 = '<p>A reminder of how things stand after round 7/24! 📊</p><p><a href="https://twitter.com/hashtag/F1" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>F1</span></a>...'
text2 = '<p>Iranian officials, state media confirm President Raisi killed in helicopter crash </p><p>The Iranian Cabinet of Ministers convened an emergency meeting on May 20 after the official confirmation...'
text3= '<p>Autolobby raus aus den Kinderzimmern 🙈</p>'

res1 = requests.post(url='http://127.0.0.1:9090/mpreprocess',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text1})

res2 = requests.post(url='http://127.0.0.1:9090/mpreprocess',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text2})

res3 = requests.post(url='http://127.0.0.1:9090/mpreprocess',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text3})

print(res1.status_code, res1.text)
print(res2.status_code, res2.text)
print(res3.status_code, res3.text)

In [ ]:
text1 = "COMP90024 is the best subject"
text2 = "I hate local computations"
text3 = "I'm studying COMP90024"
text4 = res2.text

res1 = requests.get(url='http://127.0.0.1:9090/sentiment',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text1})

res2 = requests.get(url='http://127.0.0.1:9090/sentiment',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text2})

res3 = requests.get(url='http://127.0.0.1:9090/sentiment',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text3})

res4 = requests.get(url='http://127.0.0.1:9090/sentiment',
                    headers={'Content-Type': 'application/json'},
                    json={'text': text4})

print(res1.status_code, res1.text)
print(res2.status_code, res2.text)
print(res3.status_code, res3.text)
print(res4.status_code, res4.text)

### Reformat Date

In [ ]:
date = "2024-05-20 07:39:09+00:00"
date_res = requests.post(url=f'http://127.0.0.1:9090/mredate'
                        , headers={'Content-Type': 'application/json'}
                        , json={'date': date})

print(date_res.status_code, date_res.text)

## Data Query APIs

### The concat API

In [ ]:
import requests
import pandas as pd

In [ ]:
start_date = "2024-05-17"  # start date of data query
end_date = "2024-05-19"  # end date of data query
size = 2000  # max number of entries in the concat data (optional, default is 20)
bsize = 2000  # max number of entries in the bom dataset (optional, default is 20)
msize = 2000  # max number of entries in the mastodon dataset (optional, default is 20)

res = requests.get(url=f'http://127.0.0.1:9090/concat/{start_date}/{end_date}?size={size}&bsize={bsize}&msize={msize}')

print(res.status_code, res.json())

In [ ]:
res_df = pd.DataFrame(res.json())
res_df['created_at'] = pd.to_datetime(res_df['created_at'])
res_df

In [ ]:
# continue processing...

### The BOM API
Use this api to query data in the BOM index

In [ ]:
# size parameter is used to limit the number of entries in the bom dataset
size = 20
bom_res = requests.get(url=f'http://127.0.0.1:9090/bomquery/{start_date}/{end_date}?size={size}')

print(bom_res.status_code, bom_res.json())

In [ ]:
bom_hit = pd.DataFrame(bom_res.json()['hits']['hits'])
bom_data = []
for data in bom_hit['_source']:
    bom_data.append(data)

bom_df = pd.DataFrame(bom_data)
bom_df

### The Mostodon API

In [ ]:
# size parameter is used to limit the number of entries in the bom dataset
size = 200
m_res = requests.get(url=f'http://127.0.0.1:9090/mquery/{start_date}/{end_date}?size={size}')

print(m_res.status_code, m_res.json())

In [ ]:
m_hit = pd.DataFrame(m_res.json()['hits']['hits'])
m_data = []
for data in m_hit['_source']:
    m_data.append(data)

m_df = pd.DataFrame(m_data)
m_df

### SA2 API
Use this api to retreive the median age given a `sa2_code`

In [ ]:
sa2_code = 101041024  # change this to the desired sa2 code

age_res = requests.get(url=f'http://127.0.0.1:9090/sa2/{sa2_code}/age')

print(age_res.status_code, age_res.text)
print(f'median age: {age_res.text}')